# Import libraries

In [2]:
# Load Libraries
import sys
import os
sys.path.append(os.path.abspath("../"))

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd

from functions.pyflood_functions import *
from functions.pyflood_calibration_attenuation_functions import *

# Import paths and settings from config
from config import (
    DEM_FILE,
    ALIGNED_WATER_STATIONS_SHP,
    SEA_CORE_TIF,
    CFM_WD_TIF,
    CFM_WL_TIF,
    THRESHOLD_DEPTH,
    DISTANCE_TO_COAST_TIF,
    LANDCOVER_FILE,
    ALIGNED_HWM_SHP,
    BAYESIAN_OPTIMIZATION_CSV,
    MANUAL_REDUCTION_FACTORS,
    CFM_A_WD_TIF,
    CFM_A_WL_TIF,
    CFM_A_WD_FINAL_TIF,
    CFM_A_WL_FINAL_TIF
)

## Kriging with External Drift Process and Coastal Flood Map (CFM) Generation

In [4]:
# --------------------------------------------------
# Load DEM and Aligned Stations
# --------------------------------------------------
z_dem, transform, crs = load_raster_data(DEM_FILE)
dem_lon, dem_lat, _ = extract_raster_coordinates(z_dem, transform, crs)

aligned_stations = load_aligned_water_stations(ALIGNED_WATER_STATIONS_SHP)
lon_stations = aligned_stations[:, 0]
lat_stations = aligned_stations[:, 1]
wl_stations = aligned_stations[:, 2]
dem_stations = aligned_stations[:, 3]

print(f"DEM size: {z_dem.shape}, Valid elevations: {np.count_nonzero(~np.isnan(z_dem)):,}")

# --------------------------------------------------
# Kriging Interpolation of Water Levels
# --------------------------------------------------
wl_pred, wl_var = krige_water_levels(
    lon_stations, lat_stations, wl_stations, dem_stations,
    dem_lon, dem_lat, z_dem
)
# Optional: Save raw predicted water levels and associated variance
# save_raster(wl_pred, reference_raster=DEM_FILE, output_filename="../output_data/wl_pred.tif")
# save_raster(wl_var, reference_raster=DEM_FILE, output_filename="../output_data/wl_var.tif")

Coordinates extracted in UTM.
Shapefile fields available: ['X_UTM', 'Y_UTM', 'Water_Leve', 'DEM_Elevat', 'geometry']
Loaded 71 aligned water stations.
DEM size: (24000, 24000), Valid elevations: 576,000,000
Total number of prediction points: 576000000
[Before kriging] Memory Usage: 13.12 GB | CPU Usage: 1.7%
Parallel Kriging completed in 546.54 seconds.
[After kriging] Memory Usage: 16.06 GB | CPU Usage: 2.1%


In [5]:
# --------------------------------------------------
# Compute Water Depth and Save Initial Predictions
# --------------------------------------------------
wd_pred = compute_water_depth(wl_pred, z_dem)
save_raster(wd_pred, reference_raster=DEM_FILE, output_filename="../output_data/wd_pred.tif")

# --------------------------------------------------
# Mask with Sea Core and Save Coastal Flood Maps (CFM)
# --------------------------------------------------
wd_pred_masked = mask_with_sea_core(wd_pred, SEA_CORE_TIF)
save_raster(wd_pred_masked, reference_raster=DEM_FILE, output_filename=CFM_WD_TIF)

wl_pred_masked = wd_pred_masked + z_dem
save_raster(wl_pred_masked, reference_raster=DEM_FILE, output_filename=CFM_WL_TIF)

apply_threshold_and_export(
    input_file=CFM_WD_TIF,
    threshold_value=THRESHOLD_DEPTH,
    output_suffix="_over_1m"
)

Saved raster to: ../output_data/wd_pred.tif
Saved raster to: ../output_data\CFM_Flood_Water_Depth.tif
Saved raster to: ../output_data\CFM_Flood_Water_Level.tif
Thresholded raster exported to: ../output_data\CFM_Flood_Water_Depth_over_1m.tif


## Calibration Process and Coastal Flood Map + Attenuation (CFM+A) Generation

In [8]:
# --------------------------------------------------
# Load Land Cover, Distance to Coastline, and Aligned HWMs
# --------------------------------------------------
rf_test, _, _ = load_raster_data(LANDCOVER_FILE)
distances, _, _ = load_raster_data(DISTANCE_TO_COAST_TIF)
gdf_aligned = gpd.read_file(ALIGNED_HWM_SHP)

# --------------------------------------------------
# Validate PreCCFM Predictions Against Observed HWMs
# --------------------------------------------------
gdf_comparison = compare_hwms_to_model(gdf_aligned, wl_pred_masked, crs)

observed_hwm = gdf_comparison["hwm"].values
predicted_wl = gdf_comparison["z_wl"].values
rmse, mae, scatter_index, bias, num_points = compute_error_metrics(observed_hwm, predicted_wl)
print(f"Initial RMSE: {rmse:.3f} m | MAE: {mae:.3f} m | Scatter Index: {scatter_index:.3f} | Bias: {bias:.3f} m")

initial_flood_extent_points = num_points
print(f"Total of HWMs within Flood Extent of PreCCFM: {initial_flood_extent_points}")

# --------------------------------------------------
# Bayesian Optimization of Reduction Factors
# --------------------------------------------------
# Define reduction factor bounds (min and max dictionaries)
value_mapping_min = {
    2.0: 0.000000,
    3.0: 0.000000,
    4.0: 0.000000,
    5.0: 0.000000,
    6.0: 0.000000,
    7.0: 0.000000,
    8.0: 0.000000,
    9.0: 0.000000,
    10.0: 0.000000,
    11.0: 0.000000,
    12.0: 0.000000,
    13.0: 0.000000,
    14.0: 0.000000,
    15.0: 0.000000,
    16.0: 0.000000,
    17.0: 0.000000,
    18.0: 0.000000,
    19.0: 0.000000,
    20.0: 0.000000,
    21.0: 0.000000,
    22.0: 0.000000,
    23.0: 0.000000
}


value_mapping_max = {
    2.0: 0.004000,
    3.0: 0.002667,
    4.0: 0.002000,
    5.0: 0.001000,
    6.0: 0.000400,
    7.0: 0.000250,
    8.0: 0.000250,
    9.0: 0.000714,
    10.0: 0.000571,
    11.0: 0.000571,
    12.0: 0.000500,
    13.0: 0.000250,
    14.0: 0.000278,
    15.0: 0.000278,
    16.0: 0.000250,
    17.0: 0.000278,
    18.0: 0.000278,
    19.0: 0.000115,
    20.0: 0.000115,
    21.0: 0.000125,
    22.0: 0.000125,
    23.0: 0.000125
}
pbounds = define_parameter_bounds(value_mapping_min, value_mapping_max)

if not os.path.exists(BAYESIAN_OPTIMIZATION_CSV):
    pd.DataFrame(columns=['Iteration', 'MSE', 'Num_Valid_Points'] + [f'rf_{int(k)}' for k in value_mapping_min.keys()]).to_csv(BAYESIAN_OPTIMIZATION_CSV, index=False)

from bayes_opt import BayesianOptimization

optimizer = BayesianOptimization(
    f=lambda **rf_values: objective_function_bayesopt(
        rf_values, rf_test, distances, wl_pred, z_dem, gdf_aligned, initial_flood_extent_points, BAYESIAN_OPTIMIZATION_CSV
    ),
    pbounds=pbounds
)

optimizer.maximize(init_points=22, n_iter=10)

print("Best reduction factors found:", optimizer.max)

# --------------------------------------------------
# Apply Optimized Reduction Factors
# --------------------------------------------------
best_rf = optimizer.max['params']
best_rf_fixed = {int(k.split('_')[1]): v for k, v in best_rf.items()}

reduced_wl_best, reduced_wd_best = apply_optimized_reduction_factors(
    rf_test, distances, best_rf_fixed, wl_pred_masked, z_dem
)

reduced_wl_best_fixed = np.full_like(reduced_wl_best, np.nan)
valid_mask = ~np.isnan(reduced_wd_best)
reduced_wl_best_fixed[valid_mask] = reduced_wd_best[valid_mask] + z_dem[valid_mask]
reduced_wl_best = reduced_wl_best_fixed

reduced_wl_best_masked = mask_attenuated_results(reduced_wl_best, SEA_CORE_TIF)
reduced_wd_best_masked = mask_attenuated_results(reduced_wd_best, SEA_CORE_TIF)

save_attenuated_raster(reduced_wd_best_masked, DEM_FILE, CFM_A_WD_TIF)
save_attenuated_raster(reduced_wl_best_masked, DEM_FILE, CFM_A_WL_TIF)

apply_threshold_and_export(
    input_file=CFM_A_WD_TIF,
    threshold_value=THRESHOLD_DEPTH,
    output_suffix="_over_1m"
)

# --------------------------------------------------
# Apply Manual Reduction Factors (Defined in config.py)
# --------------------------------------------------
reduced_wl_final, reduced_wd_final = apply_optimized_reduction_factors(
    rf_test, distances, MANUAL_REDUCTION_FACTORS, wl_pred_masked, z_dem
)

reduced_wl_final_fixed = np.full_like(reduced_wl_final, np.nan)
valid_mask = ~np.isnan(reduced_wd_final)
reduced_wl_final_fixed[valid_mask] = reduced_wd_final[valid_mask] + z_dem[valid_mask]
reduced_wl_final = reduced_wl_final_fixed

reduced_wl_final_masked = mask_attenuated_results(reduced_wl_final, SEA_CORE_TIF)
reduced_wd_final_masked = mask_attenuated_results(reduced_wd_final, SEA_CORE_TIF)

save_attenuated_raster(reduced_wd_final_masked, DEM_FILE, CFM_A_WD_FINAL_TIF)
save_attenuated_raster(reduced_wl_final_masked, DEM_FILE, CFM_A_WL_FINAL_TIF)

apply_threshold_and_export(
    input_file=CFM_A_WD_FINAL_TIF,
    threshold_value=THRESHOLD_DEPTH,
    output_suffix="_over_1m"
)

Comparison results saved to: hwm_vs_wl_comparison.shp
Initial RMSE: 0.520 m | MAE: 0.371 m | Scatter Index: 0.146 | Bias: 0.179 m
Total of HWMs within Flood Extent of PreCCFM: 57
|   iter    |  target   |   rf_10   |   rf_11   |   rf_12   |   rf_13   |   rf_14   |   rf_15   |   rf_16   |   rf_17   |   rf_18   |   rf_19   |   rf_2    |   rf_20   |   rf_21   |   rf_22   |   rf_23   |   rf_3    |   rf_4    |   rf_5    |   rf_6    |   rf_7    |   rf_8    |   rf_9    |
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Iteration 0: Modified MSE = 0.381091, Num_Valid_Points = 51
| 1         | -0.3811   | 0.0004961 | 0.0005563 | 0.0001342 | 0.0001049 | 0.0002359 | 0.0001384 | 0.000136  | 2.522e-05 | 0.0002118 | 5.265e-05 | 0.002934  | 5.872e-07 | 6.031e-05 |